In [1]:
import tensorflow as tf


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

Read the text from file and print the first few lines

In [3]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [4]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

**Vectorize sentences from corpus**

we can use text vectorization layer to vectorize sentences from the corpus, first few sentences above the text needs to be in one case and punctuation needs to be removed. to do this define a custom_standardisation function that can be used in text vectorization layer.

In [5]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [6]:
import re

In [7]:
import string

In [8]:
vectorize_layer.adapt(text_ds.batch(1024))

In [9]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [10]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(tf.data.AUTOTUNE).map(vectorize_layer).unbatch()

**Obtain Sequences from the dataset**

In [11]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


**Inspect few examples from the sequence**

In [12]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [13]:
import numpy as np

In [14]:
import tqdm

In [15]:
import io

# **PREPARATION OF POSITIVE AND NEGATIVE SAMPLES**

In [16]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          #seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [17]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size)

100%|██████████████████████████████████████████████████████████████████████████| 32777/32777 [00:22<00:00, 1474.65it/s]


In [18]:
targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")



targets.shape: (65208,)
contexts.shape: (65208, 5)
labels.shape: (65208, 5)


**Configure the dataset for performance**

to perform efficient batching for potentially large number of training examples, we use tf.dataset.Data API

In [19]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [20]:
dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [21]:
import tensorflow as tf
from tensorflow.keras import layers

In [22]:
num_ns = 4

# **TRAINING THE ALGORITHM**

The word2vec model can be can be implemented as a classifier to distinguish between true context of the words obtained through negative samlping.
we can perform a dot product multiplication between the embeddings of target and context words to obtain predictions for labels and compute the loss function against the true labels in the dataset.

In [23]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

Define loss function and compile model

In [24]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

Initialising word2vec class with an embedding dimension of 100, and compiling the model using Adam optimizer

In [25]:
embedding_dim = 100
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

Defining the callback to log training statistics for Tensorboard.

In [26]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


Train the model on dataset for some epochs


In [27]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


Epoch 1/20
63/63 [==============================] - 6s 37ms/step - loss: 1.6084 - accuracy: 0.2299 5s
Epoch 2/20
63/63 [==============================] - 1s 20ms/step - loss: 1.5923 - accuracy: 0.5213 1s -
Epoch 3/20
63/63 [==============================] - 1s 22ms/step - loss: 1.5518 - accuracy: 0.5623 0s - loss: 1.5522 - accuracy: 0.56
Epoch 4/20
63/63 [==============================] - 1s 18ms/step - loss: 1.4817 - accuracy: 0.5359
Epoch 5/20
63/63 [==============================] - 1s 18ms/step - loss: 1.3974 - accuracy: 0.5430
Epoch 6/20
63/63 [==============================] - 1s 22ms/step - loss: 1.3120 - accuracy: 0.5713
Epoch 7/20
63/63 [==============================] - 1s 21ms/step - loss: 1.2306 - accuracy: 0.6034
Epoch 8/20
63/63 [==============================] - 1s 20ms/step - loss: 1.1544 - accuracy: 0.6355
Epoch 9/20
63/63 [==============================] - 1s 19ms/step - loss: 1.0832 - accuracy: 0.6689
Epoch 10/20
63/63 [==============================] - 1s 22ms/step 

# **HYPER PARAMETER TUNING USING OTHER OPTIMIZERS**

In [28]:
embedding_dim = 100
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='Adamax',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [29]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


Epoch 1/20
63/63 [==============================] - 2s 24ms/step - loss: 1.6091 - accuracy: 0.2143
Epoch 2/20
63/63 [==============================] - 1s 9ms/step - loss: 1.6063 - accuracy: 0.3045
Epoch 3/20
63/63 [==============================] - 1s 10ms/step - loss: 1.6030 - accuracy: 0.3863
Epoch 4/20
63/63 [==============================] - 1s 14ms/step - loss: 1.5982 - accuracy: 0.4546
Epoch 5/20
63/63 [==============================] - 1s 14ms/step - loss: 1.5916 - accuracy: 0.5001
Epoch 6/20
63/63 [==============================] - 1s 10ms/step - loss: 1.5829 - accuracy: 0.5250
Epoch 7/20
63/63 [==============================] - 1s 10ms/step - loss: 1.5722 - accuracy: 0.5351
Epoch 8/20
63/63 [==============================] - 1s 15ms/step - loss: 1.5599 - accuracy: 0.5352
Epoch 9/20
63/63 [==============================] - 1s 13ms/step - loss: 1.5464 - accuracy: 0.5321
Epoch 10/20
63/63 [==============================] - 1s 8ms/step - loss: 1.5319 - accuracy: 0.5281
Epoch 11/20

In [30]:
embedding_dim = 100
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='Rmsprop',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [31]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 4s 50ms/step - loss: 1.6080 - accuracy: 0.2371
Epoch 2/20
63/63 [==============================] - 1s 16ms/step - loss: 1.5935 - accuracy: 0.4724
Epoch 3/20
63/63 [==============================] - 1s 14ms/step - loss: 1.5662 - accuracy: 0.5281
Epoch 4/20
63/63 [==============================] - 1s 17ms/step - loss: 1.5268 - accuracy: 0.5269
Epoch 5/20
63/63 [==============================] - 1s 14ms/step - loss: 1.4803 - accuracy: 0.5266
Epoch 6/20
63/63 [==============================] - 1s 16ms/step - loss: 1.4311 - accuracy: 0.5321 0s - loss: 1.4377 
Epoch 7/20
63/63 [==============================] - 1s 19ms/step - loss: 1.3818 - accuracy: 0.5428 0s - loss:
Epoch 8/20
63/63 [==============================] - 1s 18ms/step - loss: 1.3338 - accuracy: 0.5572
Epoch 9/20
63/63 [==============================] - 1s 14ms/step - loss: 1.2876 - accuracy: 0.5728
Epoch 10/20
63/63 [==============================] - 1s 14ms/step - loss: 1.243

In [32]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [33]:
len(vocab)

4096

In [34]:
len(weights)

4096

In [35]:
vocab[7]

'you'

In [36]:
weights[7]

array([ 0.2315299 ,  0.07285646, -0.1962219 , -0.0746958 , -0.29530913,
        0.09970913,  0.02069301, -0.07089131,  0.03045541, -0.04059046,
        0.05053968,  0.35808548, -0.15960693,  0.12220062,  0.20624779,
        0.08177194, -0.10249384,  0.289677  ,  0.2643164 ,  0.15399735,
       -0.00762504, -0.0556525 ,  0.05075707, -0.321245  ,  0.1341709 ,
        0.14681524,  0.04757319, -0.08633887, -0.14138676, -0.00646457,
       -0.26118568,  0.11109001,  0.08909423, -0.30923563,  0.08244026,
       -0.05547729, -0.1717562 , -0.16588567, -0.08822463, -0.09726227,
       -0.05205543, -0.33618903, -0.05267168,  0.2861286 ,  0.18018207,
        0.11939853,  0.00803809, -0.22336781, -0.20535085,  0.10816341,
        0.08245791,  0.4003676 ,  0.0989694 ,  0.09789625,  0.14066122,
        0.17471457,  0.02469319, -0.29544437,  0.05013512, -0.11290441,
       -0.19149028, -0.08292069, -0.14902122,  0.2598031 , -0.05147489,
        0.20342366, -0.0071078 , -0.21004862,  0.04492521, -0.03

In [37]:
weights[0].shape

(100,)